# ```Asana``` API Tutorial #

### What is ```Asana``` API? ###
```Asana``` API makes it possible to extend or enhance what’s possible with Asana's UI, integrating with other tools, adding or extracting data, or automating workflows. 

### Instalation ###
```Asana``` API is accessed via HTTP requests, so no need for installs.

### Authentication Token ###
The first thing we need to do is to authorize our user in Asana's API. To do so, we must visit the [Developer Console](https://app.asana.com/0/my-apps) and create a Personal Access Token (PAT). Our token is stored in the ```asana_token.txt```.

### Workspace GID ###
A ```workspace``` is the highest unit in Asana. Since all objects (```tasks```, ```projects```, ```teams```, etc) have an associated ```workspace```, we need to know the Globally Unique Identifier (workspace GID). To do so, log in, then visit (```GET``` endpoint) ```https://app.asana.com/api/1.0/workspaces``` in a browser.

### Main Objects ###
- ```Portfolio```: gives a high-level overview of the status of multiple ```projects```;
- ```Project```: represents a prioritized list of ```tasks``` and exists in a single ```workspace```;
- ```Task```: is Asana's basic object and can be configured with ```sections``` and ```memberships```;
- ```Users```: represents an account in Asana that can be given access to various ```workspaces```, ```projects```, and ```tasks```;
- ```Workspaces```: the highest-level organizational unit in Asana, and all ```projects``` and ```tasks``` are associated with one;

### Accessing Response Fields ###
If not set in ```opt_fields```, responses will return the bare minimum on whatever objects are being accessed. Additionally, this fields accepts the dot operator (```.```) to drill down an objects parameters (such as ```owner.name```, for example).

### Documentation ###
- Asana Guide Documentation - [Link](https://developers.asana.com/docs)
- Asana API Reference: [Link](https://developers.asana.com/reference)
---

### Importing Dependencies ###

In [45]:
import yaml
import json
import requests
from datetime import datetime

### User-Defined Classes ###

In [46]:
class HttpRequests(object):
    def __init__(self):
        return None
    
    def status_check(self, request_obj, has_payload=True, is_json=True):
        # Pulling status code
        status = request_obj.status_code
        # Confirming the use case
        if 100 <= status < 200: # Informational: server acknowledges and is processing the request
            print("Informational Error")
            raise(Exception(request_obj.text))
        elif 200 <= status < 300: # Success: server successfully received, understood, and processed the request
            print("Success")
            # Returning the payload
            if has_payload:
                if is_json:
                    return request_obj.json() # [json]
                else:
                    return request_obj.text # [str]
            else:
                return None
        elif 300 <= status < 400: # Redirection: server received the request, but redirect it to somewhere else
            print("Redirection Error")
            raise(Exception(request_obj.text))
        elif 400 <= status < 500: # Client Error: server couldn’t reach the endpoint
            print("Client Error")
            raise(Exception(request_obj.text))
        elif status >= 500: # Server Error: client made a valid request, but server failed to complete
            print("Client Error")
            raise(Exception(request_obj.text))
        else:
            print("Unknown Error")
            raise(Exception(request_obj.text))
        return None

In [140]:
class AsanaAPI(HttpRequests):
    def __init__(self, yaml_dict):
        # Initializing the parent class
        HttpRequests.__init__(self)
        # Attributes
        self.workspace_id = yaml_dict.get("asana",{}).get("workspace_gid")
        self.base_url = yaml_dict.get("asana",{}).get("base_url")
        self.headers = self.build_headers(yaml_dict)

    # Headers Methods
    def build_headers(self, yaml_dict):
        headers = {"Authorization": "Bearer " + yaml_dict.get("asana",{}).get("personal_token"),
                   "Content-Type": "application/json",
                   "Accept": "application/json"}
        return headers

    # Project Methods
    def get_projects(self, fields="archived,completed,completed_at,created_at,due_date,name,owner.name,permalink_url,start_on"):
        # Creating the URL
        url = self.base_url + "/workspaces/{}/projects".format(self.workspace_id)
        params = {"opt_fields": fields,
                  "archived": False}
        # Requesting
        request = requests.get(url, headers=self.headers, params=params)
        payload = self.status_check(request)
        return [{"project_name": each.get("name"),
                 "project_id": each.get("gid"),
                 "is_archieved": each.get("archived"),
                 "is_complete": each.get("completed"),
                 "created_at": each.get("created_at"),
                 "started_at": each.get("start_on"),
                 "due_at": each.get("due_date"),
                 "completed_at": each.get("completed_at"),
                 "url": each.get("permalink_url"),
                 "owner": each.get("owner",{}).get("name")} for each in payload.get("data")]

    def summary_tasks(self, project_id, fields="num_tasks,num_incomplete_tasks,num_completed_tasks"): 
        # Creating the URL
        url = self.base_url + "/projects/{}/task_counts".format(project_id)
        params = {"opt_fields": fields}
        # Requesting
        request = requests.get(url, headers=self.headers, params=params)
        payload = self.status_check(request)["data"]
        return {"tasks": payload.get("num_tasks"),
                "completed_tasks": payload.get("num_completed_tasks"),
                "incomplete_tasks": payload.get("num_incomplete_tasks")} # Opportunity: project completion percentage

    def get_project_status(self, project_id, fields="created_at,modified_at,status_type"):
        # Creating the URL
        url = self.base_url + "/status_updates"
        params = {"opt_fields": fields,
                  "parent": project_id}
        # Requesting
        request = requests.get(url, headers=self.headers, params=params)
        payload = self.status_check(request)["data"][0]
        return {"project_id": payload.get("gid"),
                "created_at": payload.get("created_at"),
                "modified_at": payload.get("modified_at"),
                "status": payload.get("status_type")}

    # Task Methods
    def get_tasks(self, project_id, fields="assignee.name,completed,completed_at,due_at,name,modified_at,permalink_url"): 
        # Creating the URL
        url = self.base_url + "/projects/{}/tasks".format(project_id)
        params = {"opt_fields": fields,
                  "completed_since": None} # 'completed_since': return incomplete or tasks that have been completed since
        # Requesting
        request = requests.get(url, headers=self.headers, params=params)
        payload = self.status_check(request)
        return [{"task_name": each.get("name"),
                 "task_id": each.get("gid"),
                 "assignee": each.get("assignee",{}).get("name"),
                 "is_complete": each.get("completed"),
                 "completed_at": each.get("completed_at"),
                 "due_at": each.get("due_at"),
                 "modified_at": each.get("modified_at"),
                 "url": each.get("permalink_url")} for each in payload.get("data")]

    def get_subtasks(self, task_id, fields="assignee.name,completed,completed_at,due_at,name,modified_at,permalink_url"): 
        # Creating the URL
        url = self.base_url + "/tasks/{}/subtasks".format(task_id)
        params = {"opt_fields": fields} 
        # Requesting
        request = requests.get(url, headers=self.headers, params=params)
        payload = self.status_check(request)
        return [{"subtask_name": each.get("name"),
                 "subtask_id": each.get("gid"),
                 "assignee": each.get("assignee",{}).get("name"),
                 "is_complete": each.get("completed"),
                 "completed_at": each.get("completed_at"),
                 "due_at": each.get("due_at"),
                 "modified_at": each.get("modified_at"),
                 "url": each.get("permalink_url")} for each in payload.get("data")]

    # User Methods
    def get_users(self, fields="name,email,uri"):
        # Creating the URL
        url = self.base_url + "/users"
        params = {"opt_fields": fields,
                  "workspace": self.workspace_id,
                  "limit": 3}  # 'limit' will need 'offset' param when in production
        # Requesting
        request = requests.get(url, headers=self.headers, params=params)
        payload = self.status_check(request)
        return [{"user_name": each.get("name"),
                 "user_id": each.get("gid"),
                 "email": each.get("email")} for each in payload.get("data")]

### Main ###

In [141]:
# Reading the YAML config
with open("./asana.yaml", "r") as _stream:
    yaml_config = yaml.safe_load(_stream)

In [142]:
# Instantiating the 'AsanaAPI' class
asana = AsanaAPI(yaml_config)

**Workspace**

In [91]:
# Get all projects in a workspace
asana.get_projects()

Success


[{'project_name': 'Project 1',
  'project_id': '1206082638344023',
  'is_archieved': False,
  'is_complete': False,
  'created_at': '2023-12-01T22:25:50.369Z',
  'started_at': None,
  'due_at': '2023-12-24',
  'completed_at': None,
  'url': 'https://app.asana.com/0/1206082638344023/1206082638344023',
  'owner': 'Bruno Vasconcelos Montoni'}]

**Project**

In [92]:
# Getting a project's tasks summary (using project_id = "1206082638344023")
asana.summary_tasks("1206082638344023")

Success


{'tasks': 2, 'completed_tasks': 0, 'incomplete_tasks': 2}

In [143]:
# Getting a project's status  (using project_id = "1206082638344023")
asana.get_project_status("1206082638344023")

Success


{'project_id': '1206082638344044',
 'created_at': '2023-12-01T23:10:16.886Z',
 'modified_at': '2023-12-01T23:10:17.035Z',
 'status': 'on_track'}

**Task**

In [112]:
# Listing a project's tasks
asana.get_tasks("1206082638344023")

Success


[{'task_name': 'Task 1',
  'task_id': '1206082638344026',
  'assignee': 'Bruno Vasconcelos Montoni',
  'is_complete': False,
  'completed_at': None,
  'due_at': None,
  'modified_at': '2023-12-01T23:09:43.289Z',
  'url': 'https://app.asana.com/0/1206082638344023/1206082638344026'},
 {'task_name': 'Task 2',
  'task_id': '1206082638344028',
  'assignee': 'Bruno Vasconcelos Montoni',
  'is_complete': False,
  'completed_at': None,
  'due_at': None,
  'modified_at': '2023-12-01T23:09:48.018Z',
  'url': 'https://app.asana.com/0/1206082638344023/1206082638344028'}]

In [114]:
# Listing a task's subtasks (using task_id = "1206082638344026")
asana.get_subtasks("1206082638344026")

Success


[{'subtask_name': 'Subtask 1.1',
  'subtask_id': '1206082638344032',
  'assignee': 'Bruno Vasconcelos Montoni',
  'is_complete': False,
  'completed_at': None,
  'due_at': None,
  'modified_at': '2023-12-02T21:54:39.331Z',
  'url': 'https://app.asana.com/0/1206082638344032/1206082638344032'},
 {'subtask_name': 'Subtask 1.2',
  'subtask_id': '1206082638344033',
  'assignee': 'Bruno Vasconcelos Montoni',
  'is_complete': False,
  'completed_at': None,
  'due_at': None,
  'modified_at': '2023-12-02T21:54:42.141Z',
  'url': 'https://app.asana.com/0/1206082638344033/1206082638344033'}]

**User**

In [135]:
# Listing all users in workspace
asana.get_users()

Success


[{'user_name': 'Crystal Lua',
  'user_id': '858461790906974',
  'email': 'cl3427@columbia.edu'},
 {'user_name': 'Alison Ewing',
  'user_id': '862353314856450',
  'email': 'aee2112@columbia.edu'},
 {'user_name': 'eps2147@columbia.edu',
  'user_id': '865589706372969',
  'email': 'eps2147@columbia.edu'}]

---